In [2]:
import fastf1
import fastf1.plotting
import pandas as pd 
import numpy as np
import re
from matplotlib import pyplot as plt
print(fastf1.__version__)
from fastf1 import utils
from fastf1.core import Laps
import fastf1.plotting
import matplotlib.pyplot as plt


fastf1.plotting.setup_mpl(misc_mpl_mods=False)

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


3.0.7


In [3]:
#Load Session
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year=2024
wknd="Abu dhabi"
ses="Q"
session = fastf1.get_session(year,wknd,ses)
session.load()

core           INFO 	Loading data for Abu Dhabi Grand Prix - Qualifying [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '55', '27', '1', '10', '63', '14', '77', '11', '22', '30', '18', '16', '20', '23', '24', '44', '43', '61']


In [4]:
#Results
results = session.results
print(results)
#results.to_csv('ResultsDutch.csv')

   DriverNumber BroadcastName Abbreviation DriverId         TeamName  \
4             4      L NORRIS          NOR                   McLaren   
1             1  M VERSTAPPEN          VER           Red Bull Racing   
55           55       C SAINZ          SAI                   Ferrari   
16           16     C LECLERC          LEC                   Ferrari   
81           81     O PIASTRI          PIA                   McLaren   
63           63     G RUSSELL          RUS                  Mercedes   
10           10       P GASLY          GAS                    Alpine   
14           14      F ALONSO          ALO              Aston Martin   
20           20   K MAGNUSSEN          MAG              Haas F1 Team   
11           11       S PEREZ          PER           Red Bull Racing   
22           22     Y TSUNODA          TSU                        RB   
27           27  N HULKENBERG          HUL              Haas F1 Team   
31           31        E OCON          OCO                    Al

In [ ]:


fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

year = 2023
ses = "Q"
num_races = 5  # Number of races to consider

# Initialize DataFrame
gapQ = pd.DataFrame(columns=['Driver', 'Team', 'Avg Gap to Pole'])

for race_num in range(1, num_races + 1):
    session = fastf1.get_session(year, race_num, ses)
    session.load()

    # Calculate fastest laps for each driver in the race
    list_fastest_laps = []
    drivers = pd.unique(session.laps['Driver'])
    for drv in drivers:
        drvs_fastest_lap = session.laps.pick_driver(drv).pick_fastest()
        list_fastest_laps.append(drvs_fastest_lap)

    fastest_laps = fastf1.core.Laps(list_fastest_laps).sort_values(by='LapTime').reset_index(drop=True)
    pole_lap = fastest_laps.iloc[0]  # Get the pole position lap

    # Calculate and store gap to pole for each driver in the race
    for index, row in fastest_laps.iterrows():
        gap = (row['LapTime'] - pole_lap['LapTime']).total_seconds()  # Calculate gap to pole in seconds
        pos = {'Driver': row['Driver'], 'Team': row['Team'], 'Gap': gap}
        gapQ = gapQ.append(pos, ignore_index=True)

# Calculate average gap to pole for each driver
avg_gap_per_driver = gapQ.groupby('Driver')['Gap'].mean().reset_index()
avg_gap_per_driver.rename(columns={'Gap': 'Avg Gap to Pole'}, inplace=True)

# Print and save results
avg_gap_per_driver.to_csv("AvgGap2023.csv", index=False)
avg_gap_per_driver


In [7]:
#QUALY
driver="SAI"
lap=session.laps.pick_driver(driver).pick_fastest()
print (lap)

#Telemetry
x = lap.telemetry['X']              
y = lap.telemetry['Y']
speed = lap.telemetry['Speed']
tel=pd.DataFrame({'X': x, 'Y': y,'Speed': speed })

tel.to_csv('SAIQAustin.csv')

Time                      0 days 01:05:35.813000
Driver                                       SAI
DriverNumber                                  55
LapTime                   0 days 00:01:32.652000
LapNumber                                   16.0
Stint                                        5.0
PitOutTime                                   NaT
PitInTime                                    NaT
Sector1Time               0 days 00:00:25.090000
Sector2Time               0 days 00:00:37.061000
Sector3Time               0 days 00:00:30.501000
Sector1SessionTime        0 days 01:04:28.251000
Sector2SessionTime        0 days 01:05:05.312000
Sector3SessionTime        0 days 01:05:35.813000
SpeedI1                                    241.0
SpeedI2                                    192.0
SpeedFL                                    212.0
SpeedST                                    330.0
IsPersonalBest                              True
Compound                                    SOFT
TyreLife            

In [8]:
#Dif in sectors 

import pandas as pd


# Select two drivers for comparison
driver1 = 'NOR'
driver2 = 'VER'

# Retrieve the fastest lap for each driver
fastest_lap_driver1 = session.laps.pick_driver(driver1).pick_fastest()
fastest_lap_driver2 = session.laps.pick_driver(driver2).pick_fastest()

# Extract sector times for the fastest laps of each driver
fastest_sector_times_driver1 = {
    'Sector1': fastest_lap_driver1['Sector1Time'].total_seconds(),
    'Sector2': fastest_lap_driver1['Sector2Time'].total_seconds(),
    'Sector3': fastest_lap_driver1['Sector3Time'].total_seconds()
}

fastest_sector_times_driver2 = {
    'Sector1': fastest_lap_driver2['Sector1Time'].total_seconds(),
    'Sector2': fastest_lap_driver2['Sector2Time'].total_seconds(),
    'Sector3': fastest_lap_driver2['Sector3Time'].total_seconds()
}

# Calculate the difference in time for each sector between the fastest laps of the two drivers
sector_time_difference = {
    'Sector1': fastest_sector_times_driver1['Sector1'] - fastest_sector_times_driver2['Sector1'],
    'Sector2': fastest_sector_times_driver1['Sector2'] - fastest_sector_times_driver2['Sector2'],
    'Sector3': fastest_sector_times_driver1['Sector3'] - fastest_sector_times_driver2['Sector3']
}

# Print the difference in time for each sector
print("Difference in time for each sector between {} and {}:".format(driver1, driver2))
for sector, difference in sector_time_difference.items():
    print("{}: {:.3f} seconds".format(sector, difference))


Difference in time for each sector between NOR and VER:
Sector1: 0.124 seconds
Sector2: -0.099 seconds
Sector3: -0.056 seconds


In [ ]:
#pickSpecificlap
gapQ = pd.DataFrame(columns=['Driver', 'Gap'])
driver = 'LEC'
laps = session.laps.pick_driver(driver)
lap=laps[laps['LapNumber']==18].iloc[0]
print(lap)
#Telemetry
x = lap.telemetry['X']              
y = lap.telemetry['Y']
speed = lap.telemetry['Speed']
tel=pd.DataFrame({'X': x, 'Y': y,'Speed': speed })
tel.to_csv("LECQBAH.csv")

In [5]:
# Gap Qualy

# Initialize DataFrame
gapQ = pd.DataFrame(columns=['Driver', 'Team', 'Gap'])
drivers = pd.unique(session.laps['Driver'])
list_fastest_laps = []

# Calculate fastest laps for each driver
for drv in drivers:
    drvs_fastest_lap = session.laps.pick_driver(drv).pick_fastest()
    list_fastest_laps.append(drvs_fastest_lap)

# Sort the laps by LapTime to determine the pole lap
fastest_laps = fastf1.core.Laps(list_fastest_laps).sort_values(by='LapTime').reset_index(drop=True)
pole_lap = fastest_laps.iloc[0]  # Get the pole position lap

# Calculate and store gap in seconds
gap_list = []
for index, row in fastest_laps.iterrows():
    gap = (row['LapTime'] - pole_lap['LapTime']).total_seconds()  # Calculate gap in seconds
    gap_list.append({'Driver': row['Driver'], 'Team': row['Team'], 'Gap': gap})

# Convert the list of dictionaries into a DataFrame
gapQ = pd.DataFrame(gap_list)

# Print the results
# gapQ.to_csv("GapAustin.csv", index=False)
gapQ


,Driver,Team,Gap
0,NOR,McLaren,0.000
1,PIA,McLaren,0.209
2,SAI,Ferrari,0.229
3,HUL,Haas F1 Team,0.291
4,VER,Red Bull Racing,0.350
5,GAS,Alpine,0.389
6,RUS,Mercedes,0.537
7,ALO,Aston Martin,0.601
8,BOT,Kick Sauber,0.609
9,PER,Red Bull Racing,0.669


In [10]:
#IDEAL laps gap drivers

# Assuming `session` and its `laps` attribute are already defined

# Extract unique drivers
drivers = pd.unique(session.laps['Driver'])

# Initialize an empty DataFrame for ideal laps
ideallaps = pd.DataFrame(columns=['Driver', 'Lap'])

# Calculate ideal lap times for each driver
for driver in drivers:
    laps = session.laps.pick_driver(driver)
    laps = laps.dropna(subset=['Sector1Time', 'Sector2Time', 'Sector3Time'])
    
    FS1 = min(laps['Sector1Time'])
    FS2 = min(laps['Sector2Time'])
    FS3 = min(laps['Sector3Time'])
    
    ILap = FS1 + FS2 + FS3
    lap = {'Driver': driver, 'Lap': ILap}
    ideallaps = ideallaps.append(lap, ignore_index=True)

# Determine the fastest ideal lap time
fastestlap = min(ideallaps['Lap'])

# Calculate the gap between each driver's ideal lap time and the fastest ideal lap time
idealgap = pd.DataFrame(columns=['Driver', 'Gap'])
for driver in drivers:
    gap = ideallaps[ideallaps['Driver'] == driver]['Lap'].iloc[0] - fastestlap
    gap_seconds = gap.total_seconds()  # Convert gap to seconds
    gapQI = {'Driver': driver, 'Gap': gap_seconds}
    idealgap = idealgap.append(gapQI, ignore_index=True)

# Print or save the 'idealgap' DataFrame
print(idealgap)
idealgap.to_csv("idealgapAustin.csv", index=False)


/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2100431162.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ideallaps = ideallaps.append(lap, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2100431162.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ideallaps = ideallaps.append(lap, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2100431162.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ideallaps = ideallaps.append(lap, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2100431162.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

   Driver    Gap
0     SAI  0.143
1     LEC    0.3
2     VER    0.0
3     NOR  0.017
4     PIA  0.591
5     HAM  1.841
6     RUS  0.625
7     MAG   0.79
8     ALO  0.917
9     TSU  1.179
10    HUL  0.809
11    GAS    0.5
12    LAW  1.026
13    ALB  1.654
14    STR  1.379
15    PER   0.65
16    OCO  1.135
17    BOT  1.763
18    COL  1.749
19    ZHO  1.915


/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2100431162.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ideallaps = ideallaps.append(lap, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2100431162.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ideallaps = ideallaps.append(lap, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2100431162.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ideallaps = ideallaps.append(lap, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2100431162.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

In [13]:
import pandas as pd

# Assuming `session` and its `laps` attribute are already defined

# Extract unique drivers
drivers = pd.unique(session.laps['Driver'])

# Initialize DataFrames for ideal laps and fastest laps
ideallaps = pd.DataFrame(columns=['Driver', 'IdealLap'])
fastestlaps = pd.DataFrame(columns=['Driver', 'FastestLap'])

# Calculate ideal lap times and fastest lap times for each driver
for driver in drivers:
    # Get the laps for the current driver
    laps = session.laps.pick_driver(driver)
    
    # Drop laps with missing sector times
    laps = laps.dropna(subset=['Sector1Time', 'Sector2Time', 'Sector3Time'])
    
    # Calculate ideal lap (sum of best sector times)
    FS1 = min(laps['Sector1Time'])
    FS2 = min(laps['Sector2Time'])
    FS3 = min(laps['Sector3Time'])
    ILap = FS1 + FS2 + FS3
    
    # Calculate fastest lap
    fastest_lap_time = min(laps['LapTime'])
    
    # Append results to respective DataFrames
    ideallaps = ideallaps.append({'Driver': driver, 'IdealLap': ILap}, ignore_index=True)
    fastestlaps = fastestlaps.append({'Driver': driver, 'FastestLap': fastest_lap_time}, ignore_index=True)

# Merge both DataFrames on the 'Driver' column
laps_comparison = pd.merge(ideallaps, fastestlaps, on='Driver')

# Calculate the gap between the fastest lap and the ideal lap for each driver
laps_comparison['Gap'] = (laps_comparison['FastestLap'] - laps_comparison['IdealLap']).dt.total_seconds()

# Print or save the DataFrame
print(laps_comparison[['Driver', 'Gap']])
#laps_comparison.to_csv("laps_comparison_gap.csv", index=False)


/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/1534041418.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ideallaps = ideallaps.append({'Driver': driver, 'IdealLap': ILap}, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/1534041418.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fastestlaps = fastestlaps.append({'Driver': driver, 'FastestLap': fastest_lap_time}, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/1534041418.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ideallaps = ideallaps.append({'Driver': driver, 'IdealLap': ILap}, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/15340

   Driver    Gap
0     SAI  0.196
1     LEC  0.127
2     VER  0.048
3     NOR  0.000
4     PIA  0.046
5     HAM  0.000
6     RUS  0.036
7     MAG  0.371
8     ALO  0.079
9     TSU  0.014
10    HUL  0.422
11    GAS  0.205
12    LAW  0.000
13    ALB  0.084
14    STR  0.067
15    PER  0.057
16    OCO  0.149
17    BOT  0.076
18    COL  0.000
19    ZHO  0.000


/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/1534041418.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ideallaps = ideallaps.append({'Driver': driver, 'IdealLap': ILap}, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/1534041418.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fastestlaps = fastestlaps.append({'Driver': driver, 'FastestLap': fastest_lap_time}, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/1534041418.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ideallaps = ideallaps.append({'Driver': driver, 'IdealLap': ILap}, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/15340

In [ ]:
#Ideal gaps TEAMS
# Assuming `session` and its `laps` attribute are already defined

# Extract unique teams
teams = pd.unique(session.laps['Team'])

# Initialize an empty DataFrame for ideal laps
ideallaps = pd.DataFrame(columns=['Team', 'Lap'])

# Calculate ideal lap times for each team
for team in teams:
    laps = session.laps[session.laps['Team'] == team]
    laps = laps.dropna(subset=['Sector1Time', 'Sector2Time', 'Sector3Time'])
    
    FS1 = min(laps['Sector1Time'])
    FS2 = min(laps['Sector2Time'])
    FS3 = min(laps['Sector3Time'])
    
    ILap = FS1 + FS2 + FS3
    lap = {'Team': team, 'Lap': ILap}
    ideallaps = ideallaps.append(lap, ignore_index=True)

# Determine the fastest ideal lap time
fastestlap = min(ideallaps['Lap'])

# Calculate the gap between each team's ideal lap time and the fastest ideal lap time
idealgap = pd.DataFrame(columns=['Team', 'Gap'])
for team in teams:
    gap = ideallaps[ideallaps['Team'] == team]['Lap'].iloc[0] - fastestlap
    gap_seconds = gap.total_seconds()  # Convert gap to seconds
    gapQI = {'Team': team, 'Gap': gap_seconds}
    idealgap = idealgap.append(gapQI, ignore_index=True)

# Print or save the 'idealgap' DataFrame
print(idealgap)
idealgap.to_csv("idealgapsTAustin.csv", index=False)


In [14]:

bestsector = pd.DataFrame(columns=['Driver', 'GapSector1', 'GapSector2', 'GapSector3'])

# Find the fastest overall sector times
fastest_sector_times = {
    'Sector1': session.laps['Sector1Time'].min(),
    'Sector2': session.laps['Sector2Time'].min(),
    'Sector3': session.laps['Sector3Time'].min()
}

for driver in drivers:
    laps = session.laps.pick_driver(driver)
    laps = laps.dropna(subset=['Sector1Time', 'Sector2Time', 'Sector3Time'])
    
    FS1 = min(laps['Sector1Time'], default=pd.Timestamp.max).total_seconds()
    FS2 = min(laps['Sector2Time'], default=pd.Timestamp.max).total_seconds()
    FS3 = min(laps['Sector3Time'], default=pd.Timestamp.max).total_seconds()
    
    gap_to_fastest_S1 = FS1-fastest_sector_times['Sector1'].total_seconds()
    gap_to_fastest_S2 = FS2-fastest_sector_times['Sector2'].total_seconds()
    gap_to_fastest_S3 = FS3-fastest_sector_times['Sector3'].total_seconds()
    
    sector_data = {
        'Driver': driver,
        'GapSector1': gap_to_fastest_S1,
        'GapSector2': gap_to_fastest_S2,
        'GapSector3': gap_to_fastest_S3
    }
    
    bestsector = bestsector.append(sector_data, ignore_index=True)

print(bestsector)
bestsector.to_csv("bestsectorAustin.csv", index=False)

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2503950700.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bestsector = bestsector.append(sector_data, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2503950700.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bestsector = bestsector.append(sector_data, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2503950700.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bestsector = bestsector.append(sector_data, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2503950700.py:29: FutureWarning: The frame.append method is deprecated and will be removed f

   Driver GapSector1 GapSector2 GapSector3
0     SAI      0.151      0.097       0.05
1     LEC      0.275      0.168      0.012
2     VER        0.0      0.099      0.056
3     NOR      0.172        0.0        0.0
4     PIA       0.43      0.161      0.155
5     HAM      0.447      0.924      0.625
6     RUS      0.403      0.141      0.236
7     MAG      0.357      0.342      0.246
8     ALO      0.441      0.399      0.232
9     TSU      0.335      0.559       0.44
10    HUL      0.523      0.328      0.113
11    GAS      0.245       0.33       0.08
12    LAW      0.411      0.446      0.324
13    ALB      0.429      0.833      0.547
14    STR      0.567      0.409      0.558
15    PER      0.251      0.248      0.306
16    OCO       0.57      0.394      0.326
17    BOT      0.634      0.677      0.607
18    COL      0.697      0.704      0.503
19    ZHO      0.711      0.776      0.583


/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2503950700.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bestsector = bestsector.append(sector_data, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2503950700.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bestsector = bestsector.append(sector_data, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2503950700.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bestsector = bestsector.append(sector_data, ignore_index=True)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_48115/2503950700.py:29: FutureWarning: The frame.append method is deprecated and will be removed f

In [ ]:
import pandas as pd
import fastf1

# Enable cache
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

# Initialize DataFrame
gapQ = pd.DataFrame(columns=['Team', 'Gap'])

# Bahrain Grand Prix for 2023 and 2024
gpbahrain_2023 = "Saudi Arabia"
gpbahrain_2024 = "Saudi Arabia"

# Qualifying sessions for 2023 and 2024
ses_2023 = 'Q'
ses_2024 = 'Q'

# Get session data for 2023
session_2023 = fastf1.get_session(2023, gpbahrain_2023, ses_2023)
session_2023.load()

# Get session data for 2024
session_2024 = fastf1.get_session(2024, gpbahrain_2024, ses_2024)
session_2024.load()

# Calculate fastest lap time for each team in 2023
team_fastest_laps_2023 = session_2023.laps.groupby('Team')['LapTime'].min()

# Calculate fastest lap time for each team in 2024
team_fastest_laps_2024 = session_2024.laps.groupby('Team')['LapTime'].min()

# Update team names if they changed
team_fastest_laps_2023 = team_fastest_laps_2023.rename(index={'Alfa Romeo': 'Kick Sauber', 'AlphaTauri': 'RB'})
team_fastest_laps_2024 = team_fastest_laps_2024.rename(index={'Alfa Romeo': 'Kick Sauber', 'AlphaTauri': 'RB'})

# Calculate and store gap in seconds for each team
for team in team_fastest_laps_2023.index:
    if team in team_fastest_laps_2024.index:
        gap_2023 = team_fastest_laps_2023[team].total_seconds()
        gap_2024 = team_fastest_laps_2024[team].total_seconds()
        gap = gap_2023 - gap_2024
        gapQ = gapQ.append({'Team': team, 'Gap': gap}, ignore_index=True)

# Print and save results
print(gapQ)
gapQ.to_csv("timegainedSaudi.csv", index=False)

In [ ]:
import pandas as pd
import fastf1

# Enable cache
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")  # Add your cache directory if necessary

# Initialize DataFrame
gapQ = pd.DataFrame(columns=['Driver', 'Team', 'Gap'])

year = 2023
gp = "Bahrain"
ses = 'Q'

# Get session data
session = fastf1.get_session(year, gp, ses)
session.load()
drivers = pd.unique(session.laps['Driver'])
list_fastest_laps = []

# Calculate fastest laps for each driver
for drv in drivers:
    drvs_fastest_lap = session.laps.pick_driver(drv).pick_fastest()
    list_fastest_laps.append(drvs_fastest_lap)

fastest_laps = fastf1.core.Laps(list_fastest_laps)

# Group fastest laps by team
team_fastest_laps = fastest_laps.groupby('Team').min()

# Calculate pole position lap for each team
team_pole_laps = team_fastest_laps.sort_values(by='LapTime').reset_index()

# Calculate and store gap in seconds
for index, row in fastest_laps.iterrows():
    team_pole_lap = team_pole_laps[team_pole_laps['Team'] == row['Team']].iloc[0]
    gap = (row['LapTime'] - team_pole_lap['LapTime']).total_seconds()  # Calculate gap in seconds
    pos = {'Driver': row['Driver'], 'Team': row['Team'], 'Gap': gap}
    gapQ = gapQ.append(pos, ignore_index=True)

# Print and save results
print(gapQ)
